Ejercicio: árboles de decisión y arquitectura modelo

Nuestro objetivo en este ejercicio es utilizar un clasificador de árbol de decisiones para predecir si un delito individual se resolverá, en función de información simple como dónde tuvo lugar y qué tipo de delito fue.

Visualización de datos
Como de costumbre, comencemos cargando y echando un vistazo a nuestros datos:

In [ ]:
import pandas
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/san_fran_crime.csv

#Import the data from the .csv file
dataset = pandas.read_csv('san_fran_crime.csv', delimiter="\t")

#Let's have a look at the data and the relationship we are going to model
print(dataset.head())
print(dataset.shape)

Nuestros datos parecen ser una combinación de variables categóricas como Categoría delictiva o PdDistrict, y variables numéricas como el día_del_año (1-365) y el tiempo_en_horas (hora del día, convertida en flotante). También tenemos X e Y que se refieren a coordenadas GPS y Resolución que es nuestra etiqueta.

Visualicemos nuestros datos:

In [ ]:
import graphing # custom graphing code. See our GitHub repo for details
import numpy as np

# Crime category
graphing.multiple_histogram(dataset, label_x='Category', label_group="Resolution", histfunc='sum', show=True)

# District
graphing.multiple_histogram(dataset, label_group="Resolution", label_x="PdDistrict", show=True)

# Map of crimes
graphing.scatter_2D(dataset, label_x="X", label_y="Y", label_colour="Resolution", title="GPS Coordinates", size_multiplier=0.8, show=True)

# Day of the week
graphing.multiple_histogram(dataset, label_group="Resolution", label_x="DayOfWeek", show=True)

# day of the year
# For graphing we simplify this to week or the graph becomes overwhelmed with bars
dataset["week_of_year"] = np.round(dataset.day_of_year / 7.0)
graphing.multiple_histogram(dataset, 
                    label_x='week_of_year',
                    label_group='Resolution',
                    histfunc='sum', show=True)
del dataset["week_of_year"]

Siempre vale la pena inspeccionar sus datos antes de sumergirse. Lo que podemos ver aquí es que:

La mayoría de los delitos denunciados no se resolvieron en 2016<br>
Diferentes distritos policiales informaron diferentes volúmenes de delincuencia.<br>
Diferentes distritos policiales informaron diferentes tasas de éxito en la resolución de delitos.<br>
El viernes y el sábado típicamente tenían más delitos que otros días.<br>
Larsony/Theft fue abrumadoramente el delito más común denunciado<br>
Finalización de la preparación de datos
Finalicemos nuestra preparación de datos mediante la codificación one-hot de nuestras características categóricas:

In [ ]:
# One-hot encode categorical features
dataset = pandas.get_dummies(dataset, columns=["Category", "PdDistrict"], drop_first=False)

print(dataset.head())

También necesitamos hacer un conjunto de entrenamiento y prueba.

¿Notaste con cuántos datos estábamos trabajando antes? Si no es así, vuelva a comprobar las impresiones desde arriba.

Tenemos más de 150.000 muestras con las que trabajar. Esa es una gran cantidad de datos. Debido al gran tamaño, podemos darnos el lujo de tener una mayor proporción en el conjunto de entrenamiento que normalmente tendríamos.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset in an 90/10 train/test ratio. 
# We can afford to do this here because our dataset is very very large
# Normally we would choose a more even ratio
train, test = train_test_split(dataset, test_size=0.1, random_state=2, shuffle=True)

print("Data shape:")
print("train", train.shape)
print("test", test.shape)

Código de evaluación del modelo
Ajustaremos varios modelos aquí, por lo que para maximizar la reutilización del código, crearemos un método dedicado que entrene un modelo y luego lo pruebe.

Nuestra etapa de prueba utiliza una métrica llamada "precisión equilibrada", a la que nos referiremos como "precisión" para abreviar a lo largo de este ejercicio. No es crítico que entiendas esta métrica para estos ejercicios, pero en esencia está entre 0 y 1:

0 significa que ninguna respuesta fue correcta<br>
1 significa que todas las respuestas fueron correctas<br>
La precisión equilibrada tiene en cuenta que nuestro conjunto de datos tiene más delitos sin resolver que resueltos. Cubriremos lo que esto significa en material de aprendizaje posterior en este curso.

In [ ]:
from sklearn.metrics import balanced_accuracy_score

# Make a utility method that we can re-use throughout this exercise
# To easily fit and test out model

features = [c for c in dataset.columns if c != "Resolution"]


def fit_and_test_model(model):
    '''
    Trains a model and tests it against both train and test sets
    '''  
    global features

    # Train the model
    model.fit(train[features], train.Resolution)

    # Assess its performance
    # -- Train
    predictions = model.predict(train[features])
    train_accuracy = balanced_accuracy_score(train.Resolution, predictions)

    # -- Test
    predictions = model.predict(test[features])
    test_accuracy = balanced_accuracy_score(test.Resolution, predictions)

    return train_accuracy, test_accuracy


print("Ready to go!")

Ajuste de un árbol de decisión
Usemos un árbol de decisiones para ayudarnos a determinar si se resolverá un delito. Los árboles de decisión son modelos de categorización que dividen las decisiones en varios pasos. Se pueden comparar con un diagrama de flujo, en el que se toma una decisión en cada nivel subsiguiente del árbol.

In [ ]:
import sklearn.tree

# fit a simple tree using only three levels
model = sklearn.tree.DecisionTreeClassifier(random_state=2, max_depth=3) 
train_accuracy, test_accuracy = fit_and_test_model(model)

print("Model trained!")
print("Train accuracy", train_accuracy)
print("Test accuracy", test_accuracy)


¡No esta mal! Ahora que el modelo está entrenado, visualicémoslo para que podamos tener una mejor idea de cómo funciona (¡y también de dónde obtiene su nombre de árbol!):

In [ ]:
#--------------
from sklearn.tree import plot_tree
from matplotlib import pyplot as plt

plot = plt.subplots(figsize = (4,4), dpi=300)[0]
plot = plot_tree(model,
                fontsize=3,
                feature_names = features, 
                class_names = ['0','1'], # class_names in ascending numerical order 
                label="root",
                impurity=False,
                filled=True) 
plt.show()

Todos los recuadros de color azul corresponden a la predicción de que se resolvería un delito.

Eche un vistazo al árbol para ver lo que cree que es importante para predecir un resultado. Compare esto con los gráficos que hicimos anteriormente. ¿Puedes ver una relación entre los dos?

La partitura que tenemos no está mal, pero el árbol es bastante sencillo. Veamos si podemos hacerlo mejor.

Mejora del rendimiento a través de la arquitectura
Intentaremos mejorar el rendimiento de nuestro modelo cambiando su arquitectura. Centrémonos en el parámetro de profundidad_máxima.

Nuestro árbol anterior era relativamente simple y poco profundo con una profundidad_máxima = 3. Veamos qué sucede si lo aumentamos a 100:

In [ ]:
# fit a very deep tree
model = sklearn.tree.DecisionTreeClassifier(random_state=1, max_depth=100)

train_accuracy, test_accuracy = fit_and_test_model(model)
print("Train accuracy", train_accuracy)
print("Test accuracy", test_accuracy)

Como puedes imaginar, un árbol con una profundidad_máxima = 100 es grande. Demasiado grande para visualizarlo aquí, así que pasemos directamente a ver cómo funciona el nuevo modelo en nuestros datos de entrenamiento.

Tanto el entrenamiento como la precisión de las pruebas han aumentado mucho. La formación, sin embargo, ha aumentado mucho más. Si bien estamos contentos con la mejora en la precisión de las pruebas, esta es una clara señal de sobreajuste.

El sobreajuste con árboles de decisión se vuelve aún más obvio cuando tenemos conjuntos de datos de tamaño más típico (más pequeño). Volvamos a ejecutar el ejercicio anterior pero con solo 100 muestras de entrenamiento:

In [ ]:
# Temporarily shrink the training set to something
# more realistic
full_training_set = train
train = train[:100]

# fit the same tree as before
model = sklearn.tree.DecisionTreeClassifier(random_state=1, max_depth=100)

# Assess on the same test set as before
train_accuracy, test_accuracy = fit_and_test_model(model)
print("Train accuracy", train_accuracy)
print("Test accuracy", test_accuracy)

# Roll the training set back to the full set
train = full_training_set

El modelo funciona mal en los datos de prueba. Con conjuntos de datos de tamaño razonable, los árboles de decisión son notoriamente propensos al sobreajuste. En otras palabras, tienden a adaptarse muy bien a los datos en los que están capacitados, pero generalizan muy mal a datos desconocidos. Esto empeora cuanto más profundo se vuelve el árbol o más pequeño se vuelve el conjunto de entrenamiento. A ver si podemos mitigar esto.

podar un arbol
La poda es el proceso de simplificar un árbol de decisión para que proporcione los mejores resultados de clasificación y, al mismo tiempo, reduzca el sobreajuste. Hay dos tipos de poda: prepoda y pospoda.

La poda previa implica restringir el modelo durante el entrenamiento, para que no crezca más de lo útil. Cubriremos esto a continuación.

La post-poda es cuando simplificamos el árbol después de entrenarlo. No implica tomar ninguna decisión de diseño antes de tiempo, sino simplemente optimizar el modelo existente. Esta es una técnica válida pero es bastante complicada, por lo que no la cubrimos aquí debido a limitaciones de tiempo.

Prepoda
Podemos realizar una poda previa generando muchos modelos, cada uno con diferentes parámetros de profundidad máxima. Para cada uno, registramos la precisión equilibrada para el conjunto de prueba. Para mostrar que esto es importante incluso con conjuntos de datos bastante grandes, aquí trabajaremos con 10000 muestras.

In [ ]:
# Temporarily shrink the training set to 10000
# for this exercise to see how pruning is important
# even with moderately large datasets
full_training_set = train
train = train[:10000]


# Loop through the values below and build a model
# each time, setting the maximum depth to that value 
max_depth_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ,15, 20, 50, 100]
accuracy_trainset = []
accuracy_testset = []
for depth in max_depth_range:
    # Create and fit the model
    prune_model = sklearn.tree.DecisionTreeClassifier(random_state=1, max_depth=depth)

    # Calculate and record its sensitivity
    train_accuracy, test_accuracy = fit_and_test_model(prune_model)
    accuracy_trainset.append(train_accuracy)
    accuracy_testset.append(test_accuracy)

# Plot the sensitivity as a function of depth  
pruned_plot = pandas.DataFrame(dict(max_depth=max_depth_range, accuracy=accuracy_trainset))

fig = graphing.line_2D(dict(train=accuracy_trainset, test=accuracy_testset), x_range=max_depth_range, show=True)

# Roll the training set back to the full thing
train = full_training_set

Podemos ver en nuestro gráfico que la mejor precisión se obtiene para una profundidad máxima de aproximadamente 10. Estamos buscando simplificar nuestro árbol, por lo que elegimos profundidad máxima = 10 para nuestro árbol podado final:

In [ ]:
# Temporarily shrink the training set to 10000
# for this exercise to see how pruning is important
# even with moderately large datasets
full_training_set = train
train = train[:10000]


# Not-pruned
model = sklearn.tree.DecisionTreeClassifier(random_state=1)
train_accuracy, test_accuracy = fit_and_test_model(model)
print("Unpruned Train accuracy", train_accuracy)
print("Unpruned Test accuracy", test_accuracy)


# re-fit our final tree to print out its performance
model = sklearn.tree.DecisionTreeClassifier(random_state=1, max_depth=10)
train_accuracy, test_accuracy = fit_and_test_model(model)
print("Train accuracy", train_accuracy)
print("Test accuracy", test_accuracy)

# Roll the training set back to the full thing
train = full_training_set

Nuestro nuevo y mejorado modelo podado muestra una precisión equilibrada marginalmente mejor en el conjunto de prueba y un rendimiento mucho peor en el conjunto de entrenamiento que el modelo que no está podado. Esto significa que nuestra poda ha reducido significativamente el sobreajuste.

Si lo desea, regrese y cambie el número de muestras a 100, y observe cómo cambia la profundidad máxima óptima. Piense por qué podría ser esto (pista: complejidad del modelo frente al tamaño de la muestra)

Otra opción con la que te puede gustar jugar es cuántas funciones se ingresan en el árbol. Se pueden observar patrones similares de sobreajuste manipulando esto. De hecho, el número y el tipo de características proporcionadas a un árbol de decisión pueden ser incluso más importantes que su tamaño.

Resumen

En esta unidad tratamos los siguientes temas:

Usar técnicas de visualización para obtener información sobre nuestros datos<br>
Construcción de un modelo de árbol de decisión simple<br>
Uso del modelo entrenado para predecir etiquetas<br>
Recortar un árbol de decisiones para reducir los efectos del sobreajuste